In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests

## Project functions to do stuff

In [4]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    vgg_tag                    = 'vgg16'
    vgg_input_tensor_name      = 'image_input:0'
    vgg_keep_prob_tensor_name  = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph       = tf.get_default_graph()
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob   = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out  = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out  = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out  = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)

    return image_input, keep_prob, layer3_out, layer4_out, layer7_out
tests.test_load_vgg(load_vgg, tf)


def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # scale vgg out layers (as mentioned on forum)
    # vgg_layer3_out = tf.multiply(vgg_layer3_out, 0.0001, name='vgg_layer3_out_scaled')
    # vgg_layer4_out = tf.multiply(vgg_layer4_out, 0.01,   name='vgg_layer4_out_scaled')
    
    # 1x1 convs
    layer7_ones = tf.layers.conv2d(inputs             = vgg_layer7_out,
                                   filters            = num_classes,
                                   kernel_size        = (1, 1),
                                   padding            ='same',
                                   kernel_initializer = tf.random_normal_initializer(stddev=0.01),
                                   kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3))
    
    layer4_ones = tf.layers.conv2d(inputs             = vgg_layer4_out,
                                   filters            = num_classes,
                                   kernel_size        = (1, 1),
                                   padding            ='same',
                                   kernel_initializer = tf.random_normal_initializer(stddev=0.01),
                                   kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3))
    
    layer3_ones = tf.layers.conv2d(inputs             = vgg_layer3_out,
                                   filters            = num_classes,
                                   kernel_size        = (1, 1),
                                   padding            ='same',
                                   kernel_initializer = tf.random_normal_initializer(stddev=0.01),
                                   kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3))
    # upsampling
    output = tf.layers.conv2d_transpose(inputs             = layer7_ones,
                                        filters            = num_classes,
                                        kernel_size        = (4, 4),
                                        strides            = (2, 2),
                                        padding            ='same',
                                        kernel_initializer = tf.random_normal_initializer(stddev=0.01),
                                        kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3))
    # skip connection
    output = tf.add(output, layer4_ones)
    # upsample 
    output = tf.layers.conv2d_transpose(inputs             = output,
                                        filters            = num_classes,
                                        kernel_size        = (4, 4),
                                        strides            = (2, 2),
                                        padding            ='same',
                                        kernel_initializer = tf.random_normal_initializer(stddev=0.01),
                                        kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3))
    # skip connection
    output = tf.add(output, layer3_ones)
    # upsample 
    output = tf.layers.conv2d_transpose(inputs             = output,
                                        filters            = num_classes,
                                        kernel_size        = (16, 16),
                                        strides            = (8, 8),
                                        padding            ='same',
                                        kernel_initializer = tf.random_normal_initializer(stddev=0.01),
                                        kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3))
    return output
tests.test_layers(layers)


def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    logits    = tf.reshape(nn_last_layer, (-1, num_classes))
    labels    = tf.reshape(correct_label, (-1, num_classes))
    # add regularization
    loss      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)) + sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op  = optimizer.minimize(loss)
    return logits, train_op, loss
tests.test_optimize(optimize)

Tests Passed
Tests Passed
Tests Passed


In [5]:
# train func
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    log_every = 10
    for epoch in range(epochs):
        print("Epoch {}:".format(epoch+1))
        counter = 0
        for imgs, gt_imgs in get_batches_fn(batch_size):
            counter += 1
            # train
            sess.run(train_op, feed_dict={input_image:   imgs,
                                          correct_label: gt_imgs,
                                          keep_prob:     0.5,
                                          learning_rate: 1e-4})
            # check
            if counter % log_every == log_every-1:
                tr_loss = sess.run(cross_entropy_loss, feed_dict={input_image:   imgs,
                                                                  correct_label: gt_imgs,
                                                                  keep_prob:     1})
                print("Train loss =", tr_loss)

tests.test_train_nn(train_nn)

## Run function (main)

In [6]:
def run():
    num_classes = 2
    image_shape = [160, 576]
    data_dir    = './data'
    runs_dir    = './runs'
    epochs      = 1
    batch_size  = 1
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)
    
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.25)
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    # with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # Placeholders
        correct_label = tf.placeholder(dtype=tf.float32, shape=[None, image_shape[0], image_shape[1], num_classes], name='images')
        learning_rate = tf.placeholder(dtype=tf.float32, name='lr')

        # Build NN using load_vgg, layers, and optimize function
        image_input, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
        output = layers(layer3_out, layer4_out, layer7_out, num_classes)        
        logits, train_op, loss = optimize(output, correct_label, learning_rate, num_classes)
        
        # Init
        sess.run(tf.global_variables_initializer())
        
        # Train NN using the train_nn function
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, loss, 
                 image_input, correct_label, keep_prob, learning_rate)

        # Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, image_input)

        # OPTIONAL: Apply the trained model to a video

In [ ]:
%%time
run()